In [1]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt

In [2]:
import os
import dotenv

dotenv.load_dotenv()

True

In [3]:
import os
import pandas as pd
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from pprint import pprint

# os.environ["GPT_API_KEY"] = GPT_API_KEY
UPSTAGE_API_KEY = os.environ.get("UPSTAGE_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)


In [4]:
folder_path = '../data'

In [5]:
document_list = []

for file in os.listdir(folder_path):
    print(file)
    temp_loader = CSVLoader(file_path=f"{folder_path}/{file}", encoding='utf-8-sig')
    temp_document_list = temp_loader.load_and_split(text_splitter=text_splitter)
    
    document_list.extend(temp_document_list)

print(len(document_list))

CANCEL_main_qna_list.csv
DELIVERY_main_qna_list.csv
ORDER_main_qna_list.csv
REFUND_main_qna_list.csv
SUB_qna_list.csv
309


In [6]:
# Upstage 에서 제공하는 Embedding Model을 활용
embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [7]:
from langchain_pinecone import PineconeVectorStore


index_name = 'upstage-index'

c:\Users\Admin\anaconda3\envs\final_P\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:
# # DB 처음 만들 때
# database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)


In [8]:
# 만들어 놓은 DB가 있을 때
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

# Vectorstore 유사도 검색

In [9]:
query = '반품 비용은 얼마인가요?'

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")
    

* [SIM=0.484761] : 3
Question: Q[반품비] 반품 비용은 누가 부담하나요?
Answer: 귀책사유에 따라 반품 비용의 부담 주체가 다릅니다. 상품 불량/파손/오배송 등 판매자의 귀책인 경우에는 고객에게 교환/반품 비용이 발생하지 않습니다. 그러나 구매자의 단순 변심인 경우에는 고객에게 반품 비용이 부담될 수 있습니다.
 구매자 단순 변심의 경우 (클릭)
 반품 비용 확인 방법 (클릭)
keywords: CANCEL
count: 0 [{'row': 3.0, 'source': '../data/CANCEL_main_qna_list.csv'}]
* [SIM=0.480641] Question: 단순 변심 반품 비용
Answer: 로켓배송 상품
• 와우 멤버십 회원 : 무료
• 와우 멤버십 비회원
ㆍ총 주문금액 - 반품 상품금액 = 19800원 미만인 경우 반품비 5000원 차감 후 환불
ㆍ총 주문금액 - 반품 상품금액 = 19800원 이상인 경우 반품비 2500원 차감 후 환불
 로켓직구 상품
• 와우 멤버십 회원 : 5000원 (미국 직구 2600원)
• 와우 멤버십 비회원
ㆍ총 주문금액 - 반품 상품금액 = 29800원 미만인 경우 반품비 5000원 차감 후 환불 (미국 직구 2600원)
ㆍ총 주문금액 - 반품 상품금액 = 29800원 이상인 경우 반품비 2500원 차감 후 환불 (미국 직구 2500원)
• 단 TV 및 일부 전자기기와 같은 특정 제품(다이슨 로봇청소기 등)은 2600원/5000원 보다 높은 교환/반품비가 적용될 수 있음
 판매자 배송 상품
• 배송 시작 이후 최초 배송비를 포함한 왕복 배송비 차감 후 환불
• 반품비는 판매자 및 상품에 따라 상이함
• 도서산간 지역 설치 상품 업체에서 직접 배송하는 상품의 경우 반품비가 추가될 수 있음
keywords: SUB
count: 0 [{'row': 15.0, 'source': '../data/SUB_qna_list.csv'}]
* [SIM=0.475614] : 10
Question

# LLM 질의 테스트

In [15]:
%pip install pymongo pytz

Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

query = '반품 비용은 얼마인가요?'

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")



template = """
[context]: {context}
----

당신은 고객센터 상담사로서 고객의 문의에 답변하는 역할을 합니다. 항상 부드럽고 공손한 어조로 응대하며, 다음 지침을 따라주세요:

1. [context]에서 제공된 상위 3개의 정보를 바탕으로 고객에게 안내하세요.
   - 질문과 가장 적합한 정보를 우선적으로 선택하되, 나머지 정보도 필요에 따라 참조하여 답변에 포함하세요.
   - 답변은 구체적이고 부드러운 구어체로 작성합니다.
   - 상담사가 실제 대화에서 사용하는 것처럼, **"확인해본 결과"**, **"안내드리겠습니다"** 등의 문구를 활용하세요.

2. 항상 고객을 환영하거나 공감하는 **쿠션어**로 시작하세요.
   - 예: "네, 고객님. 문의 주셔서 감사합니다.", "네, 고객님. 확인해본 결과 ~"

3. **중요**: 모든 문장을 너무 짧거나 단조롭게 작성하지 말고, 부드럽게 연결하여 대화의 흐름을 유지하세요.
   - 질문에 가장 적합한 정보를 핵심으로 전달하며, 필요한 경우 다른 정보를 추가적으로 참조합니다.
   - 딱딱한 느낌을 피하고, 구체적인 정보를 자연스럽게 전달합니다.

4. 필요할 경우에만 추가 정보를 요청하세요.
   - 예: "정확한 안내를 위해 [필요 정보]를 확인 부탁드립니다."

5. 모든 답변은 2~3줄로 작성하되, 문장은 온점으로 구분하고 연결감을 유지합니다.

6. 답변이 없을 경우[SIM<=0.4], "해당 내용에 대한 정확한 정보를 찾을 수 없어 지금 답변 드리기 어려운 점 양해 부탁드립니다. 확인 해보고 연락드리겠습니다."라고 안내하세요.

### 참고 예시:
[context]: 
1. 미사용 상품에 한하여 교환/반품이 가능합니다. 상품을 회수한 후 검수 단계에서 문제가 발견될 시 고객님께 연락을 드릴 수 있습니다.
2. 파손/불량/오배송 등(판매자 귀책)의 경우 물품을 수령한 날부터 3개월 이내 또는 그 사실을 알거나 알 수 있었던 날부터 30일 이내 교환/반품이 가능합니다.
3. 로켓배송 상품은 배송 완료 후 30일 이내, 로켓모바일 상품은 주문일 포함 14일 이내에 교환/반품이 가능합니다.

질문: 반품 가능 기간이 어떻게 되나요?
답변: 네, 고객님. 문의 주셔서 감사합니다. 확인해본 결과, 단순 변심의 경우 로켓배송 상품은 배송 완료 후 30일 이내에 반품 신청이 가능합니다. 로켓모바일 상품은 주문일 포함 14일 이내에 가능합니다. 또한, 파손/불량 등 판매자 귀책의 경우 물품 수령 후 3개월 이내에 교환/반품이 가능합니다. 더 궁금한 점 있으시면 말씀 부탁드립니다!

---
[질의]: {query}

위의 [context] 정보 내에서 [질의]에 대해 상담사 입장에서 가장 관련성이 높은 메인 답변을 제공합니다.  
1. 쿠션어로 시작하여 자연스럽고 연결감 있는 구어체로 작성합니다.  
2. 항상 관련성이 높은 정보를 우선적으로 안내하되, 나머지 정보를 필요에 따라 참조합니다.  
3. 필요할 경우에만 추가 요청을 포함하며, 문장은 온점으로 구분하여 간결하게 작성합니다.

"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)

answer = chain.invoke(query).replace('  ', ' ').split('.')
print("Answer : ", end='')
for ans in answer:
    print(ans + '.')
    

* [SIM=0.484761] : 3
Question: Q[반품비] 반품 비용은 누가 부담하나요?
Answer: 귀책사유에 따라 반품 비용의 부담 주체가 다릅니다. 상품 불량/파손/오배송 등 판매자의 귀책인 경우에는 고객에게 교환/반품 비용이 발생하지 않습니다. 그러나 구매자의 단순 변심인 경우에는 고객에게 반품 비용이 부담될 수 있습니다.
 구매자 단순 변심의 경우 (클릭)
 반품 비용 확인 방법 (클릭)
keywords: CANCEL
count: 0 [{'row': 3.0, 'source': '../data/CANCEL_main_qna_list.csv'}]
* [SIM=0.480641] Question: 단순 변심 반품 비용
Answer: 로켓배송 상품
• 와우 멤버십 회원 : 무료
• 와우 멤버십 비회원
ㆍ총 주문금액 - 반품 상품금액 = 19800원 미만인 경우 반품비 5000원 차감 후 환불
ㆍ총 주문금액 - 반품 상품금액 = 19800원 이상인 경우 반품비 2500원 차감 후 환불
 로켓직구 상품
• 와우 멤버십 회원 : 5000원 (미국 직구 2600원)
• 와우 멤버십 비회원
ㆍ총 주문금액 - 반품 상품금액 = 29800원 미만인 경우 반품비 5000원 차감 후 환불 (미국 직구 2600원)
ㆍ총 주문금액 - 반품 상품금액 = 29800원 이상인 경우 반품비 2500원 차감 후 환불 (미국 직구 2500원)
• 단 TV 및 일부 전자기기와 같은 특정 제품(다이슨 로봇청소기 등)은 2600원/5000원 보다 높은 교환/반품비가 적용될 수 있음
 판매자 배송 상품
• 배송 시작 이후 최초 배송비를 포함한 왕복 배송비 차감 후 환불
• 반품비는 판매자 및 상품에 따라 상이함
• 도서산간 지역 설치 상품 업체에서 직접 배송하는 상품의 경우 반품비가 추가될 수 있음
keywords: SUB
count: 0 [{'row': 15.0, 'source': '../data/SUB_qna_list.csv'}]
* [SIM=0.475614] : 10
Question